In [1]:
# Import of required modules
import numpy as np
import pandas as pd
import datetime
import gc

In [2]:
# Customized function for loading train/test data and applying correct data transformations
def load_df_main(path, **kwargs):
    df = pd.read_csv(path, **kwargs)
    
    df["first_active_month"] = pd.to_datetime(df["first_active_month"])
    
    categories = ["feature_1", "feature_2", "feature_3"]
    for feature in categories:
        df[feature] = df[feature].astype("category")
        
    return df

In [3]:
# Load train data
train = load_df_main("../input/train.csv")

In [4]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [5]:
train.describe(include="all")

,first_active_month,card_id,feature_1,feature_2,feature_3,target
count,201917,201917,201917.0,201917.0,201917.0,201917.000000
unique,75,201917,5.0,3.0,2.0,NaN
top,2017-09-01 00:00:00,C_ID_b61fedab1b,3.0,1.0,1.0,NaN
freq,13878,1,73573.0,89242.0,114198.0,NaN
first,2011-11-01 00:00:00,NaN,NaN,NaN,NaN,NaN
last,2018-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,-0.393636
std,NaN,NaN,NaN,NaN,NaN,3.850500
min,NaN,NaN,NaN,NaN,NaN,-33.219281
25%,NaN,NaN,NaN,NaN,NaN,-0.883110


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
first_active_month    201917 non-null datetime64[ns]
card_id               201917 non-null object
feature_1             201917 non-null category
feature_2             201917 non-null category
feature_3             201917 non-null category
target                201917 non-null float64
dtypes: category(3), datetime64[ns](1), float64(1), object(1)
memory usage: 5.2+ MB


In [7]:
# Load test data
test = load_df_main("../input/test.csv")

In [8]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04-01,C_ID_0ab67a22ab,3,3,1
1,2017-01-01,C_ID_130fd0cbdd,2,3,0
2,2017-08-01,C_ID_b709037bc5,5,1,1
3,2017-12-01,C_ID_d27d835a9f,2,1,0
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1


In [9]:
test.describe(include="all")

,first_active_month,card_id,feature_1,feature_2,feature_3
count,123622,123623,123623.0,123623.0,123623.0
unique,75,123623,5.0,3.0,2.0
top,2017-09-01 00:00:00,C_ID_fa2ff19bbf,3.0,1.0,1.0
freq,8805,1,44719.0,54775.0,69770.0
first,2011-11-01 00:00:00,NaN,NaN,NaN,NaN
last,2018-01-01 00:00:00,NaN,NaN,NaN,NaN


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123623 entries, 0 to 123622
Data columns (total 5 columns):
first_active_month    123622 non-null datetime64[ns]
card_id               123623 non-null object
feature_1             123623 non-null category
feature_2             123623 non-null category
feature_3             123623 non-null category
dtypes: category(3), datetime64[ns](1), object(1)
memory usage: 2.2+ MB


In [11]:
# Customized function for loading historical_transactions / new_merchant_transactions
def load_df_transactions(path, **kwargs):
    df = pd.read_csv(path, **kwargs)
    
    # Apply correct data types to features
    df["purchase_date"] = pd.to_datetime(df["purchase_date"])
    categories = ["authorized_flag", "city_id", "category_1", "category_3", "merchant_category_id", "state_id", "subsector_id"]
    for feature in categories:
        df[feature] = df[feature].astype("category")

    strings = ["card_id", "merchant_id"]
    for feature in strings:
        df[feature] = df[feature].astype("str")

    ints = ["month_lag", "installments"]
    for feature in ints:
        df[feature] = df[feature].astype("int16")

    floats = ["purchase_amount"]
    for feature in floats:
        df[feature] = df[feature].astype("float32")
    
    # Fill missing values with most dominant value
    df["category_2"].fillna(1.0,inplace=True)
    df["category_2"] = df["category_2"].astype("str").apply(lambda x: x[:1] if x != "nan" else x).astype("category") #workaround to transform e.g. 1.0000 to 1
    df["category_3"].fillna("A",inplace=True)
    df["merchant_id"].fillna("M_ID_00a6ca8a8a",inplace=True)
    
    return df

## Feature Engineering

In [12]:
# Add new features to historical_transactions / new_merchant_transactions
def add_features_transactions(df):
    df["purchase_year"] = df["purchase_date"].dt.year
    df["purchase_weekofyear"] = df["purchase_date"].dt.weekofyear
    df["purchase_month"] = df["purchase_date"].dt.month
    df["purchase_dayofweek"] = df["purchase_date"].dt.dayofweek
    df["purchase_weekend"] = (df.purchase_date.dt.weekday >=5).astype(int)
    df["purchase_hour"] = df["purchase_date"].dt.hour
    df["authorized_flag"] = df["authorized_flag"].map({"Y":1, "N":0}) # Map into binary
    df["category_1"] = df["category_1"].map({"Y":1, "N":0}) # Map into binary
    df["month_diff"] = ((datetime.datetime.today() - df["purchase_date"]).dt.days)//30
    df["month_diff"] += df["month_lag"]
    df = pd.get_dummies(df, columns=["category_2", "category_3"]) # Get dummy variables for category
    
    return df

In [13]:
# Aggregate features from historical_transactions / new_merchant_transactions
def aggregate_transactions(df, prefix):
    
    aggs = {}
    aggs["purchase_date"] = ["max","min"]
    aggs["month_diff"] = ["mean"]
    aggs["card_id"] = ["size"]
    
    for col in ["purchase_amount", "installments", "month_lag"]:
        aggs[col] = ["sum","max","min","mean","var"]
        
    for col in ["authorized_flag", "purchase_weekend", "category_1", "category_2_1", "category_2_2", "category_2_3", "category_2_4", "category_2_5", "category_3_A", "category_3_B", "category_3_C"]:
        aggs[col] = ["sum", "mean"]
        
    for col in ["purchase_year", "purchase_weekofyear", "purchase_month", "purchase_dayofweek", "purchase_hour", "subsector_id", "merchant_id", "merchant_category_id"]:
        aggs[col] = ["nunique"]
    
    df_agg = df.groupby("card_id").agg(aggs)
    
    df_agg.columns = [prefix + "_" + "_".join(column_pair) for column_pair in df_agg.columns.ravel()] # Rename columns with respective dataframe name
    df_agg.reset_index(inplace=True)
    
    return df_agg

In [14]:
# Load historical_transactions data set
hist = load_df_transactions("../input/historical_transactions.csv")

In [15]:
hist = add_features_transactions(hist)

In [16]:
aggregated_transactions = aggregate_transactions(hist, "hist")
train = pd.merge(train, aggregated_transactions, on='card_id', how='left')
test = pd.merge(test, aggregated_transactions, on='card_id', how='left')
del aggregated_transactions # del for memory savings
del hist # del for memory savings
gc.collect()

184

In [17]:
# Load new_merchant_transactions data set
new = load_df_transactions("../input/new_merchant_transactions.csv")

In [18]:
new = add_features_transactions(new)

In [19]:
aggregated_transactions = aggregate_transactions(new, "new")
train = pd.merge(train, aggregated_transactions, on='card_id', how='left')
test = pd.merge(test, aggregated_transactions, on='card_id', how='left')
del aggregated_transactions # delete for memory savings
del new # delete for memory savings
gc.collect()

182

In [20]:
# Add additional features to the train/test data frames
def add_features_main(df):
    df["year"] = df["first_active_month"].dt.year.astype("category")
    df["weekofyear"] = df["first_active_month"].dt.weekofyear.astype("category")
    df["month"] = df["first_active_month"].dt.month.astype("category")
    df["elapsed_time"] = (datetime.date(2018, 2, 1) - df["first_active_month"].dt.date).dt.days
    
    df["hist_purchase_date_diff"] = (df["hist_purchase_date_max"] - df["hist_purchase_date_min"]).dt.days
    df["hist_purchase_date_average"] = df["hist_purchase_date_diff"]/df["hist_card_id_size"]
    df["hist_purchase_date_uptonow"] = (datetime.datetime.today() - df["hist_purchase_date_max"]).dt.days

    df["hist_first_buy"] = (df["hist_purchase_date_min"] - df["first_active_month"]).dt.days
    df["new_first_buy"] = (df["new_purchase_date_min"] - df["first_active_month"]).dt.days
    
    for col in ["hist_purchase_date_max","hist_purchase_date_min","new_purchase_date_max", "new_purchase_date_min"]:
        df[col] = df[col].astype(np.int64) * 1e-9
    df["card_id_total"] = df["new_card_id_size"] + df["hist_card_id_size"]
    df["purchase_amount_total"] = df["new_purchase_amount_sum"] + df["hist_purchase_amount_sum"]

In [21]:
add_features_main(train)
add_features_main(test)

In [22]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_purchase_date_max,hist_purchase_date_min,hist_month_diff_mean,hist_card_id_size,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_installments_sum,hist_installments_max,hist_installments_min,hist_installments_mean,hist_installments_var,hist_month_lag_sum,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_category_2_1_sum,hist_category_2_1_mean,hist_category_2_2_sum,hist_category_2_2_mean,hist_category_2_3_sum,hist_category_2_3_mean,hist_category_2_4_sum,hist_category_2_4_mean,hist_category_2_5_sum,...,new_authorized_flag_mean,new_purchase_weekend_sum,new_purchase_weekend_mean,new_category_1_sum,new_category_1_mean,new_category_2_1_sum,new_category_2_1_mean,new_category_2_2_sum,new_category_2_2_mean,new_category_2_3_sum,new_category_2_3_mean,new_category_2_4_sum,new_category_2_4_mean,new_category_2_5_sum,new_category_2_5_mean,new_category_3_A_sum,new_category_3_A_mean,new_category_3_B_sum,new_category_3_B_mean,new_category_3_C_sum,new_category_3_C_mean,new_purchase_year_nunique,new_purchase_weekofyear_nunique,new_purchase_month_nunique,new_purchase_dayofweek_nunique,new_purchase_hour_nunique,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,year,weekofyear,month,elapsed_time,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_first_buy,new_first_buy,card_id_total,purchase_amount_total
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,1.519551e+09,1.498573e+09,10.473077,260,-165.968735,2.258394,-0.739395,-0.638341,0.045003,4,1,0,0.015385,0.015206,-1017,0,-8,-3.911538,5.748901,247,0.950000,90,0.346154,0,0.000000,257.0,0.988462,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,...,1.0,6.0,0.260870,0.0,0.000000,23.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,23.0,1.000000,0.0,0.000000,0.0,0.000000,1.0,7.0,2.0,7.0,8.0,10.0,23.0,14.0,2017,22,6,245,242,0.930769,316,26,277.0,283.0,-179.212936
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,1.517438e+09,1.483720e+09,11.400000,350,-210.006332,4.630300,-0.742400,-0.600018,0.148200,543,10,-1,1.551429,2.282448,-1761,0,-12,-5.031429,14.477519,339,0.968571,132,0.377143,31,0.088571,350.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,0.000000,0.0,0.000000,6.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,6.0,1.000000,0.0,0.000000,1.0,4.0,2.0,4.0,5.0,4.0,6.0,5.0,2017,52,1,396,390,1.114286,340,5,396.0,356.0,-214.362061
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,1.519759e+09,1.484123e+09,10.325581,43,-29.167391,-0.145847,-0.730138,-0.678311,0.007635,0,0,0,0.000000,0.000000,-370,0,-13,-8.604651,14.768549,41,0.953488,11,0.255814,0,0.000000,4.0,0.093023,0.0,0.0,0.0,0.0,0.0,0.000000,39.0,...,1.0,1.0,1.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,1.0,1.0,1.000000,0.0,0.000000,0.0,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,31,8,549,412,9.581395,313,163,635.0,44.0,-29.867716
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,1.519818e+09,1.506443e+09,10.350649,77,-49.491364,1.445596,-0.740897,-0.642745,0.068447,84,3,-1,1.090909,0.346890,-218,0,-5,-2.831169,3.247437,77,1.000000,11,0.142857,12,0.155844,24.0,0.311688,0.0,0.0,0.0,0.0,53.0,0.688312,0.0,...,1.0,3.0,0.428571,1.0,0.142857,1.0,0.142857,0.0,0.0,0.0,0.000000,6.0,0.857143,0.0,0.0,1.0,0.142857,6.0,0.857143,0.0,0.000000,1.0,5.0,2.0,4.0,5.0,5.0,7.0,6.0,2017,35,9,153,154,2.000000,313,25,187.0,84.0,-54.145737
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,1.519850e+09,1.510445e+09,10.353383,133,-48.687656,7.193041,-0.746156,-0.366073,1.828159,182,12,1,1.368421,3.598086,-171,0,-3,-1.285714,1.054113,128,0.962406,42,0.315789,15,0.112782,25.0,0.187970,0.0,0.0,0.0,0.0,107.0,0.804511,1.0,...,1.0,12.0,0.333333,2.0,0.055556,4.0,0.111111,0.0,0.0,7.0,0.

In [23]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_purchase_date_max,hist_purchase_date_min,hist_month_diff_mean,hist_card_id_size,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_installments_sum,hist_installments_max,hist_installments_min,hist_installments_mean,hist_installments_var,hist_month_lag_sum,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_category_2_1_sum,hist_category_2_1_mean,hist_category_2_2_sum,hist_category_2_2_mean,hist_category_2_3_sum,hist_category_2_3_mean,hist_category_2_4_sum,hist_category_2_4_mean,hist_category_2_5_sum,hist_category_2_5_mean,...,new_authorized_flag_mean,new_purchase_weekend_sum,new_purchase_weekend_mean,new_category_1_sum,new_category_1_mean,new_category_2_1_sum,new_category_2_1_mean,new_category_2_2_sum,new_category_2_2_mean,new_category_2_3_sum,new_category_2_3_mean,new_category_2_4_sum,new_category_2_4_mean,new_category_2_5_sum,new_category_2_5_mean,new_category_3_A_sum,new_category_3_A_mean,new_category_3_B_sum,new_category_3_B_mean,new_category_3_C_sum,new_category_3_C_mean,new_purchase_year_nunique,new_purchase_weekofyear_nunique,new_purchase_month_nunique,new_purchase_dayofweek_nunique,new_purchase_hour_nunique,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,year,weekofyear,month,elapsed_time,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_first_buy,new_first_buy,card_id_total,purchase_amount_total
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,1.514510e+09,1.491330e+09,12.411765,68,-40.733734,0.235676,-0.743903,-0.599025,0.036967,141,12,1,2.073529,4.248244,-247,0,-8,-3.632353,6.026997,44,0.647059,12,0.176471,23,0.338235,68.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,1.0,1.0,0.333333,0.0,0.000000,3.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,2.0,0.666667,1.0,0.333333,1.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,2017.0,13.0,4.0,306.0,268,3.941176,374,3.0,308.0,71.0,-42.510891
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,1.518989e+09,1.484321e+09,10.461538,78,-49.136513,0.318817,-0.731881,-0.629955,0.024025,83,4,1,1.064103,0.164669,-812,0,-13,-10.410256,4.686647,77,0.987179,17,0.217949,2,0.025641,2.0,0.025641,0.0,0.0,1.0,0.012821,75.0,0.961538,0.0,0.000000,...,1.0,3.0,0.333333,2.0,0.222222,2.0,0.222222,0.0,0.0,0.0,0.000000,7.0,0.777778,0.0,0.0,0.0,0.000000,8.0,0.888889,1.0,0.111111,1.0,6.0,2.0,6.0,8.0,6.0,9.0,8.0,2017.0,52.0,1.0,396.0,401,5.141026,322,12.0,426.0,87.0,-55.081211
2,2017-08-01,C_ID_b709037bc5,5,1,1,1.517598e+09,1.503673e+09,10.846154,13,4.528841,2.525867,-0.536537,0.348372,0.821827,44,10,-1,3.384615,13.589744,-27,0,-6,-2.076923,3.076923,9,0.692308,0,0.000000,1,0.076923,2.0,0.153846,0.0,0.0,0.0,0.000000,0.0,0.000000,11.0,0.846154,...,1.0,0.0,0.000000,1.0,0.500000,1.0,0.500000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.5,0.0,0.000000,1.0,0.500000,1.0,0.500000,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2017.0,31.0,8.0,184.0,161,12.384615,339,24.0,212.0,15.0,4.708978
3,2017-12-01,C_ID_d27d835a9f,2,1,0,1.519127e+09,1.512392e+09,10.576923,26,-13.690715,0.087965,-0.731881,-0.526566,0.048032,38,6,-1,1.461538,2.178462,-32,0,-2,-1.230769,0.904615,26,1.000000,7,0.269231,0,0.000000,26.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,1.0,3.0,0.300000,1.0,0.100000,9.0,0.900000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.1,0.0,0.000000,5.0,0.500000,5.0,0.500000,1.0,7.0,2.0,5.0,7.0,8.0,10.0,10.0,2017.0,48.0,12.0,62.0,77,2.961538,321,3.0,93.0,36.0,-19.434389
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,1.519728e+09,1.483444e+09,10.536364,110,25.139385,15.782255,-0.746758,0.228540,7.715973,120,4,1,1.090909,0.193495,-685,0,-13,-6.227273,20.525855,87,0.790909,21,0.190909,0,0.000000,4.0,0.036364,0.0,0.0,5.0,0.045455,101.0,0.918182,0.0,0.000000,...,1.0,2.0,0.3333

In [ ]:
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)